# Obtención de discursos de la reina de Inglaterra desde la página web de la casa real británica. Se realiza mediante *web scraping*.


Obtenemos los enlaces a cada discurso:


In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle

base_enlaces = 'https://www.royal.uk' 
base_paginas = 'https://www.royal.uk/speeches?text=&mrf=2&date[min][date]=01/06/1909&date[max][date]=8/10/2020&id=&page='
PAGINAS = 31 # Número de paginas donde hay enlaces a discursos
 
# Obtener los enlaces de las paginas
def url_to_transcript(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    enlaces = []
    for title in soup.find_all(class_="listing"):
        for a in title.find_all('a', href=True):
            enlace_discurso = base_enlaces + a['href']
            enlaces.append(enlace_discurso) if enlace_discurso not in enlaces else enlaces
    return enlaces

urls = []
paginas = [str(i) for i in range(PAGINAS)]
for pagina in paginas:
    urls.append(base_paginas + pagina)
 
# Recorrer las URLs y obtener los enlaces página a página
enlaces_matrix = [url_to_transcript(u) for u in urls]
# Obtenemos un vector con todos los enlaces juntos
enlaces = [enlace for vector_enlaces in enlaces_matrix for enlace in vector_enlaces]


Obtenemos el texto de cada enlace, con el formato querido:

In [ ]:
def url_get_text(url):
    # Obtener los textos de los discursos.
    print('URL',url)
    text=""
    title=""
    try:
        page = requests.get(url).text
        soup = BeautifulSoup(page, "lxml")
        title = [p.text for p in soup.find_all(id="page-title-wrapper")]
        # BORRAMOS LA CITA EXTRAIDA DEL PROPIO DISCURSO
        if soup.find("blockquote") is not None:
          soup.find('blockquote').decompose()
        # BORRAMOS EL PARRAFO DE COMPARTIR
        soup.find('div', class_="tbm-share-links share").decompose()
        # Obtenemos los demás párrafos (los que nos interesan)
        text = [p.text for p in soup.find(class_="content").find_all('p')]
    except Exception:
        print('ERROR, no se pudo abrir la pagina.')
        return '', ''
    return title, text

# Recorrer las URLs y obtener los textos
textos=[]
titulos=[]
num_discursos = len(enlaces)
for i in range(len(enlaces)):
    enlace_discurso = enlaces[i]
    titulo, texto = url_get_text(enlace_discurso)
    if titulo != '':
      titulos.append(titulo)
      textos.append(texto)
    else:
      num_discursos = num_discursos - 1


Añadimos el texto a una cadena continua con saltos de línea:

In [ ]:
cad = ''
for texto in textos[2]:
    cad=cad + '\n' + texto
cad

Eliminamos los saltos de línea de los títulos:

In [ ]:
# Eliminamos los saltos de línea de los títulos
for i in range(len(titulos)):
  titulos[i][0] = ' '.join(titulos[i][0].split())

Creamos la carpeta en Drive donde guardar los ficheros con los discursos.

In [ ]:
## Creamos un directorio y nombramos los archivos por año
!rm -r /content/drive/My\ Drive/discursos_reina
!mkdir -p ./drive/My\ Drive/discursos_reina

for i in range(num_discursos):
    with open("./drive/My Drive/discursos_reina/" + str(i) + '-' + titulos[i][0] + ".txt", "w") as file:
        cad=''
        for texto in textos[i]:
                cad=cad + '\n' + texto
        file.write(cad)
        file.close()
